You can generate crystal structures for a given chemical composition partly using CrySPY  
(Here no CSP, only structure generation)  

See the document of CrySPY for details of arguments  
https://tomoki-yamashita.github.io/CrySPY/input_file.html

# Import

In [1]:
# ---------- set the path of your CrySPY
#            check /your_cryspy_path/CrySPY/gen_struc
import sys
sys.path.append('/your/cryspy_path/CrySPY')

In [2]:
import gen_struc as gs

In [3]:
# ---------- for data preparetin in EA
import pickle

# Random structure generation

## Rnd_struc_gen class  
``` python
Rnd_struc_gen(natot, atype, nat, minlen, maxlen, dangle, mindist, maxcnt=50, symprec=0.001)  
```

There are two ways  
  1) with space group infromation (symmetry)  
  2) without space gruop information

In [4]:
# ---------- example of Si4O8
natot = 12
atype = ['Si', 'O']
nat = [4, 8]
minlen = 4
maxlen = 10
dangle = 20.0
mindist = [[1.8, 1.3],
           [1.3, 1.5]]

In [5]:
# ---------- instantiate Rnd_struc_gen class
rsg = gs.random.Rnd_struc_gen(natot, atype, nat, minlen, maxlen, dangle, mindist)

## When you use space group information (symmetry) for structure generation

find_wy program is required  
tmp_gen_struc directory is temporaryly made in the current directory

In [6]:
# ---------- setting 
nstruc = 5                    # number of structures
spgnum = 'all'                # all space gruop (default: all)
# spgnum = [100, 150]         # if you want to use only space group 100 and 150
id_offset = 0                 # Structure ID starts from id_offset (default: 0)
#init_pos_path = None         # if you specify a file path,
init_pos_path = './POSCARS'   #    structure data in POSCAR format are written in the file
                              #    you can open this file using VESTA

In [7]:
# ---------- path of find_wy
fwpath = '/path/to/find_wy'

In [8]:
# ---------- generate structure
rsg.gen_with_spg(nstruc, spgnum, id_offset, init_pos_path, fwpath=fwpath)

Structure ID      0 was generated. Space group: 130 --> 129 P4/nmm
Structure ID      1 was generated. Space group: 139 --> 139 I4/mmm
Structure ID      2 was generated. Space group: 193 --> 191 P6/mmm
Structure ID      3 was generated. Space group:  16 -->  47 Pmmm
Structure ID      4 was generated. Space group:  69 -->  69 Fmmm


You can access generated structure data to see rsg.init_struc_data  
rsg.init_sturc_data is dict type like {0: struc_data_0, 1: struc_data_1, ... , ID: struc_data_ID}

In [9]:
# ---------- e.g. ID 0
rsg.init_struc_data[0]

Structure Summary
Lattice
    abc : 8.5363921716 8.5363921716 9.5472082748
 angles : 90.0 90.0 90.0
 volume : 695.7049839941718
      A : 8.5363921716 0.0 0.0
      B : 0.0 8.5363921716 0.0
      C : 0.0 0.0 9.5472082748
PeriodicSite: Si (0.0000, 0.0000, 2.3868) [0.0000, 0.0000, 0.2500]
PeriodicSite: Si (4.2682, 4.2682, 2.3868) [0.5000, 0.5000, 0.2500]
PeriodicSite: Si (4.2682, 4.2682, 7.1604) [0.5000, 0.5000, 0.7500]
PeriodicSite: Si (0.0000, 0.0000, 7.1604) [0.0000, 0.0000, 0.7500]
PeriodicSite: O (0.0000, 4.2682, 5.3848) [0.0000, 0.5000, 0.5640]
PeriodicSite: O (4.2682, 0.0000, -0.6112) [0.5000, 0.0000, -0.0640]
PeriodicSite: O (4.2682, 0.0000, -5.3848) [0.5000, 0.0000, -0.5640]
PeriodicSite: O (0.0000, 4.2682, 10.1584) [0.0000, 0.5000, 1.0640]
PeriodicSite: O (0.0000, 4.2682, 3.4059) [0.0000, 0.5000, 0.3567]
PeriodicSite: O (4.2682, 0.0000, 1.3677) [0.5000, 0.0000, 0.1433]
PeriodicSite: O (4.2682, 0.0000, -3.4059) [0.5000, 0.0000, -0.3567]
PeriodicSite: O (0.0000, 4.2682, 8.1795) [

## When you DO NOT use space group information (symmetry) for structure generation

find_wy program is NOT required  

In [10]:
# ---------- setting 
nstruc = 5                    # number of structures you will generate
spgnum = 0                    # no symmetry
id_offset = 0                 # Structure ID starts from id_offset (default: 0)
#init_pos_path = None         # if you specify a file path,
init_pos_path = './POSCARS'   #    structure data in POSCAR format are written in the file
                              #    you can open this file using VESTA

In [11]:
# ---------- generate structure
rsg.gen_wo_spg(nstruc, id_offset, init_pos_path)

Structure ID      0 was generated. Space group:   1 P1
Structure ID      1 was generated. Space group:   1 P1
Structure ID      2 was generated. Space group:   1 P1
Structure ID      3 was generated. Space group:   1 P1
Structure ID      4 was generated. Space group:   1 P1


You can access generated structure data to see rsg.init_struc_data  
rsg.init_sturc_data is dict type like {0: struc_data_0, 1: struc_data_1, ... , ID: struc_data_ID}

In [12]:
# ---------- e.g. ID 3
rsg.init_struc_data[3]

Structure Summary
Lattice
    abc : 6.298986085616132 6.298986085616132 4.425396479289587
 angles : 90.0 90.0 90.0
 volume : 175.58745495078747
      A : 6.298986085616132 0.0 0.0
      B : 3.857016573811756e-16 6.298986085616132 0.0
      C : 2.7097738166599795e-16 2.7097738166599795e-16 4.425396479289587
PeriodicSite: Si (3.9884, 2.1561, 3.7157) [0.6332, 0.3423, 0.8396]
PeriodicSite: Si (1.4341, 2.0907, 0.0809) [0.2277, 0.3319, 0.0183]
PeriodicSite: Si (1.7770, 0.2442, 1.8197) [0.2821, 0.0388, 0.4112]
PeriodicSite: Si (4.4552, 4.9589, 4.2943) [0.7073, 0.7873, 0.9704]
PeriodicSite: O (1.5869, 5.7924, 0.0311) [0.2519, 0.9196, 0.0070]
PeriodicSite: O (1.1559, 2.1744, 1.4088) [0.1835, 0.3452, 0.3183]
PeriodicSite: O (3.2611, 1.8654, 1.8974) [0.5177, 0.2961, 0.4288]
PeriodicSite: O (6.1176, 5.2960, 1.2416) [0.9712, 0.8408, 0.2806]
PeriodicSite: O (3.7168, 3.7673, 0.2011) [0.5901, 0.5981, 0.0455]
PeriodicSite: O (0.7541, 5.5006, 3.0234) [0.1197, 0.8732, 0.6832]
PeriodicSite: O (2.3278, 5.8

# Structure generation by EA
Here, let us use sample data of Si16 (sample_data_Si16_for_EA) as parent data

Si16, 10 structures

## Load parent data

In [18]:
def load_pkl(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [19]:
# ---------- load optimized structure data
opt_struc_data = load_pkl('./sample_data_Si16_for_EA/pkl_data/opt_struc_data.pkl')

In [20]:
# ---------- load rslt_data for energy datta of parent structure
rslt_data = load_pkl('./sample_data_Si16_for_EA/pkl_data/rslt_data.pkl')
rslt_data    # pandas DataFrame

,Spg_num,Spg_sym,Spg_num_opt,Spg_sym_opt,E_eV_atom,Magmom,Opt
0,214,I4_132,229,Im-3m,-4.055381,NaN,done
1,25,Pmm2,59,Pmmn,-3.891756,NaN,done
2,174,P-6,174,P-6,-3.598403,NaN,done
3,99,P4mm,99,P4mm,-1.680160,NaN,done
4,146,R3,148,R-3,-4.217574,NaN,done


In [21]:
# ---------- get energy data as dict: key is structure ID
fitness = rslt_data['E_eV_atom'].to_dict()

In [22]:
fitness

{0: -4.0553805695082525,
 1: -3.891756223526888,
 2: -3.5984027917691264,
 3: -1.680160412683843,
 4: -4.217573557286639}

If you fail to load the "rslt_data.pkl" file owing to difference version of pandas or something,  
use the following cell

In [24]:
#fitness = {
# 0: -4.0553805695082525,
# 1: -3.891756223526888,
# 2: -3.5984027917691264,
# 3: -1.680160412683843,
# 4: -4.217573557286639
#}

## Select parents
There are two ways:  
  1) Tournament selection  
  2) Roulette selection  

Here let us try tournament selection

In [26]:
# ---------- setting
elite_struc = None     # You can add additional parents,
elite_fitness = None   #     if you add elite_struc and elite fitness
fit_reverse = False    # False means minimum search
n_fittest = 5          # number of survival of the fittest

In [27]:
# ---------- instantiate Select_parents class
sp = gs.EA.Select_parents(opt_struc_data, fitness, elite_struc, elite_fitness,
                          fit_reverse, n_fittest)

Remove duplicated data
Survival of the fittest: top 5 structures survive


In this setting stable 5 (=n_fittest) structures can become candidates to be selected  
You can check survivors

In [28]:
# ---------- top 5 structures
sp.ranking_dedupe

[4, 0, 1, 2, 3]

In [29]:
# ---------- setting for tournament selection
t_size = 2
sp.set_tournament(t_size)

## Generate offspring

In [30]:
# ---------- setting
symprec = 0.001  # precision for symmetry (default: 0.001)
id_start = 5    # next structure ID (here we already have up to 4)
init_pos_path = './POSCARS'

In [31]:
# ---------- instantiate EA_generation class
eagen = gs.EA.EA_generation(sp, symprec, id_start, init_pos_path)

In [32]:
# --------- setting for crossover and strain
atype = ['Si']
nat = [16]
mindist = [[1.5]]
maxcnt_ea = 100    # default value

## Crossover
Let us make 5 structures by crosover operation

In [33]:
# ---------- setting for crossover
crs_lat = 'equal'    # 'eaual' or 'random'
crs_func = 'OP'      # one point crossover or two point crossover (TP)
nat_diff_tole = 4    # default

In [34]:
# ---------- instantiate Crossover class
co = gs.EA.Crossover(atype, nat, mindist, crs_lat, crs_func,
               nat_diff_tole, maxcnt_ea)

In [35]:
# ---------- generate
eagen.gen_crossover(n_crsov=5, co=co)

Structure ID      5 was generated from      0 and      4 by crossover. Space group:   1 P1
Structure ID      6 was generated from      2 and      0 by crossover. Space group:   1 P1
Structure ID      7 was generated from      0 and      4 by crossover. Space group:   1 P1
Structure ID      8 was generated from      1 and      4 by crossover. Space group:   1 P1
Structure ID      9 was generated from      0 and      1 by crossover. Space group:   1 P1


You can see generated structue data in eagen.offspring (dict)

In [36]:
# ---------- e.g. ID 8
eagen.offspring[8]

Structure Summary
Lattice
    abc : 5.127427802749827 6.903917483461704 8.996179916486279
 angles : 89.93028643440407 89.80024143620818 89.89679333732393
 volume : 318.4561322187947
      A : 5.127057159448769 0.04295435467375591 0.04422306605830746
      B : -0.04578507099681585 6.903622425502047 0.04447192178477887
      C : -0.04583059794762615 -0.04730780765402335 8.995938784663766
PeriodicSite: Si (2.9734, 4.0545, 8.0060) [0.5931, 0.5897, 0.8841]
PeriodicSite: Si (4.0015, 4.5270, 0.4302) [0.7866, 0.6511, 0.0407]
PeriodicSite: Si (2.9963, 0.6027, 7.9838) [0.5931, 0.0897, 0.8841]
PeriodicSite: Si (0.6605, 5.4814, 4.9075) [0.1408, 0.7968, 0.5409]
PeriodicSite: Si (0.3466, 0.5475, 0.7581) [0.0691, 0.0795, 0.0835]
PeriodicSite: Si (1.6469, 3.9988, 2.9494) [0.3293, 0.5794, 0.3234]
PeriodicSite: Si (3.2301, 1.0107, 3.5342) [0.6348, 0.1451, 0.3890]
PeriodicSite: Si (4.2720, 5.9169, 3.1148) [0.8439, 0.8541, 0.3379]
PeriodicSite: Si (2.9716, 2.4657, 0.9237) [0.5836, 0.3542, 0.0981]
Periodic

## Permutation
we skip permutaion because of one element system

In [ ]:
# # ---------- setting
# n_times = 1    # default

In [ ]:
# # ---------- instantiate Permutation class
# pm = gs.EA.Permutation(atype, mindist, ntimes, maxcnt_ea)

In [ ]:
#eagen.gen_permutation(nperm=4, pm=pm)

## Strain
Let us make 5 structures by strain

In [37]:
# ---------- setting for strain
sigma_st = 0.5    # Standard deviation for strain

In [38]:
# ---------- instantiate Strain class
st = gs.EA.Strain(atype, mindist, sigma_st, maxcnt_ea)

In [39]:
# ---------- generate
eagen.gen_strain(n_strain=5, st=st)

Structure ID     10 was generated from      0 by strain. Space group:   2 P-1
Structure ID     11 was generated from      4 by strain. Space group:   2 P-1
Structure ID     12 was generated from      1 by strain. Space group:   2 P-1
Structure ID     13 was generated from      1 by strain. Space group:   2 P-1
Structure ID     14 was generated from      1 by strain. Space group:   2 P-1


You can see generated structue data in eagen.offspring (dict)